Library


In [ ]:
import math
import joblib
import numpy as np
import pandas as pd
import scipy.signal
import scipy.fftpack
import librosa
import soundfile as sf
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt
from scipy.signal import resample_poly
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf

Main Path


In [ ]:
MAIN_PATH = r"D:\Kuliah\Matkul\Semester 4\DASAR KECERDASAN ARTIFICIAL (DKA)\[2] Tugas\[3] Tugas Besar\Dasar-Kecerdasan-Artificial_Tugas-Besar"

Konstanta


In [ ]:
SAMPLE_RATE = 48000
BATCH_SIZE = 32
SEGMENT_DURATION = int(1 * SAMPLE_RATE)
OVERLAP_DURATION = int(0.5 * SAMPLE_RATE)

Random Seed


In [ ]:
RANDOM_SEED = 21

try:
    from sklearn.utils import check_random_state
    random_state = check_random_state(RANDOM_SEED)
except ImportError:
    pass

# Prapemrosesan


In [ ]:
def load_file_audio(path):
    audio, sr = sf.read(path)
    return np.array(audio), sr

def prapemrosesan_downmixing(audio):
    if audio.ndim > 1:
        audio = np.mean(audio, axis=1)
    return audio.astype(np.float32)

def prapemrosesan_resampling(audio, sr):
    if sr == SAMPLE_RATE:
        return audio.copy(), SAMPLE_RATE
    
    ratio = SAMPLE_RATE / sr
    n_samples = int(np.round(len(audio) * ratio))
    
    x_old = np.linspace(0, 1, len(audio))
    x_new = np.linspace(0, 1, n_samples)
    return np.interp(x_new, x_old, audio), SAMPLE_RATE

def prapemrosesan_padding(audio):
    if np.mod(audio.shape[0], SEGMENT_DURATION) != 0:
        padding = SEGMENT_DURATION - (audio.shape[0] % SEGMENT_DURATION)
        audio = np.pad(audio, (0, padding))
    return audio

def prapemrosesan_splitting(audio):
    num_segments = int(np.floor((len(audio) - SEGMENT_DURATION) / OVERLAP_DURATION)) + 1
    segments = []

    for i in range(num_segments):
        start = int(i * OVERLAP_DURATION)
        end = int(start + SEGMENT_DURATION)
        segment = audio[start:end]
        if len(segment) < SEGMENT_DURATION:
            segment = np.pad(segment, (0, SEGMENT_DURATION - len(segment)), mode='constant')
        segments.append(segment)

    return np.array(segments)

# Ekstraksi Fitur


In [ ]:
def get_rms(segment):
    return np.sqrt(np.mean(segment ** 2))

def get_zcr(segment):
    return np.sum(np.abs(np.diff(np.signbit(segment)))) / (len(segment) / SAMPLE_RATE)

def get_lms(segment):
    mel_spec = librosa.feature.melspectrogram(y=segment, sr=SAMPLE_RATE)
    return np.mean(mel_spec)

# Dataset


Raw


In [ ]:
df = pd.read_excel(f"{MAIN_PATH}/dataset_labeled.xlsx")

Prapemrosesan


In [ ]:
def extract_lms_features(lms_column):
    lms_features = lms_column.apply(lambda x: np.array(eval(x)))
    return np.stack(lms_features)

Ekstraksi Fitur (LMS)


In [ ]:
lms_features = extract_lms_features(df['lms'])

Normalisasi, Split, dan Batching


In [ ]:
scaler = StandardScaler()
df[['rms', 'zcr']] = scaler.fit_transform(df[['rms', 'zcr']])

X = np.hstack([df[['rms', 'zcr']].values, lms_features])
y = df['label'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

train_dataset = train_dataset.shuffle(buffer_size=1000).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

for features, labels in train_dataset.take(1):
    print("Features shape:", features.shape)
    print("Labels shape:", labels.shape)

# LGBM


Inisialisasi


In [ ]:
lgbm_model = LGBMClassifier(random_state=21)

training


In [ ]:
# Mau pake grid search, random search, atau optima? atau ga usah aja yak.
# F : Kalo masih ada waktu sama nilainya masih jelek bisa pakai si bang.

params = {
    'objective': 'multiclass',
    'metric': 'multi_logloss',
    'num_class': 3,
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.001,
    'feature_fraction': 0.9
}



# CNN


Inisialisasi


In [ ]:
class CNN(nn.Module):
    def __init__(self, num_classes=10):
        super(CNN, self).__init__()
        test_input = np.random.rand(SAMPLE_RATE * SEGMENT_DURATION)
        lms = get_lms(test_input)
        in_channels = 1
        self.conv1 = nn.Conv2d(in_channels, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear(256, 256)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, num_classes)
    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.gap(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

training


# Conformer


Inisialisasi


In [ ]:
config = ConformerConfig(
    hidden_size=144,
    num_attention_heads=4,
    intermediate_size=576,
    conv_kernel_size=32,
    num_hidden_layers=8,
    input_feat_per_channel=n_mels,
    input_channels=1,
    max_position_embeddings=time_steps
)

conformer_model = TFConformerModel(config)

input_layer = tf.keras.layers.Input(shape=(n_mels, time_steps))
expand_dim = tf.keras.layers.Reshape((1, n_mels, time_steps))(input_layer)
conv_proj = tf.keras.layers.Conv2D(
    filters=config.hidden_size,
    kernel_size=(3, 3),
    padding='same',
    activation='relu'
)(expand_dim)
squeeze_dim = tf.keras.layers.Reshape((n_mels, time_steps, config.hidden_size))(conv_proj)
conformer_output = conformer_model(squeeze_dim).last_hidden_state
gap = tf.keras.layers.GlobalAveragePooling1D()(conformer_output)
output = tf.keras.layers.Dense(num_classes, activation='softmax')(gap)

conformer_model = tf.keras.Model(inputs=input_layer, outputs=output)

training


# Evaluasi


LGBM


CNN


Conformer
